### ECMC Project

In [ ]:
import pandas as pd, numpy as np, seaborn as sns, tensorflow as tf, xgboost as xgb, warnings, os, sys, time, datetime, gc
from sklearn.model_selection import train_test_split as tts, StratifiedKFold as skf, GridSearchCV as gscv, cross_val_score as cvs, validation_curve as vc, learning_curve as lc
from sklearn.preprocessing import StandardScaler as ss, OneHotEncoder as ohe, LabelEncoder as le, PolynomialFeatures as pf, MinMaxScaler as mms, RobustScaler as rs, PowerTransformer as pt, QuantileTransformer as qt
from sklearn.compose import ColumnTransformer as ct, make_column_selector as mcs, make_column_transformer as mct
from sklearn.pipeline import Pipeline as pl, make_pipeline as mp
from sklearn.linear_model import LogisticRegression as lr, Ridge as rdg, Lasso as lso, ElasticNet as en
from sklearn.ensemble import RandomForestClassifier as rfc, GradientBoostingClassifier as gbc, StackingClassifier as sc, VotingClassifier as vc_cls, ExtraTreesClassifier as etc, AdaBoostClassifier as abc
from sklearn.svm import SVC as svc, LinearSVC as lsvc
from sklearn.impute import SimpleImputer as si, KNNImputer as ki
from sklearn.feature_selection import SelectKBest as skb, f_classif as fc, mutual_info_classif as mic, RFE as rfe, SelectFromModel as sfm
from sklearn.metrics import classification_report as cr, confusion_matrix as cm, ConfusionMatrixDisplay as cmd, roc_curve as rc, auc as auc_fn, accuracy_score as acc, precision_score as ps, recall_score as rs_score, f1_score as f1, log_loss as ll, roc_auc_score as ras
from sklearn.decomposition import PCA as pca, TruncatedSVD as tsvd
from sklearn.manifold import TSNE as tsne
from imblearn.over_sampling import SMOTE as sm, ADASYN as ad, BorderlineSMOTE as bsm, SVMSMOTE as ssm
from imblearn.under_sampling import RandomUnderSampler as rus, EditedNearestNeighbours as enn, TomekLinks as tl
from imblearn.combine import SMOTETomek as smt, SMOTEENN as sen
from imblearn.pipeline import Pipeline as ipl
from tensorflow.keras import layers as lyr, models as mdl, optimizers as opt, regularizers as reg, callbacks as cb, utils as ku
from tensorflow.keras.callbacks import EarlyStopping as es, ReduceLROnPlateau as rlp, ModelCheckpoint as mcp
import matplotlib.pyplot as plt, matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap as lscm, ListedColormap as licm
import joblib, xlsxwriter, pickle, json, hashlib, itertools, functools, collections, copy
from scipy import stats as sp_stats
from scipy.spatial.distance import pdist as pd_dist, squareform as sq_form
from scipy.cluster.hierarchy import dendrogram as dend, linkage as link
warnings.filterwarnings('ignore')

plt.rcParams.update({k: v for k, v in [('font.family', 'Arial'), ('font.size', 11), ('font.weight', 'bold'), ('axes.linewidth', 1.2), ('axes.labelsize', 14), ('axes.labelweight', 'bold'), ('axes.titlesize', 16), ('axes.titleweight', 'bold'), ('xtick.major.width', 1.2), ('ytick.major.width', 1.2), ('xtick.major.size', 5), ('ytick.major.size', 5), ('xtick.labelsize', 12), ('ytick.labelsize', 12), ('legend.frameon', False), ('legend.fontsize', 11), ('figure.dpi', 600), ('savefig.dpi', 600), ('savefig.bbox', 'tight'), ('savefig.pad_inches', 0.1)]})

fp_base = 'C:\\Users\\anisi\\OneDrive - Colorado School of Mines\\Documents\\PhD\\PhD Research\\ECMC\\Spills_Data.xlsx'
fp_main = os.path.join(fp_base, 'Spills_Data.xlsx')
dt_hash = hashlib.md5(fp_main.encode()).hexdigest()[:8]
start_time = time.time()

df_raw = pd.read_excel(fp_main)
df_backup = df_raw.copy(deep=True)
df_work = df_raw.copy(deep=True)

# Data structure analysis
dt_info = {'shape': df_work.shape, 'columns': list(df_work.columns), 'dtypes': df_work.dtypes.to_dict(), 'missing': df_work.isnull().sum().to_dict()}
numeric_cols = df_work.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df_work.select_dtypes(include=['object']).columns.tolist()
datetime_cols = df_work.select_dtypes(include=['datetime64']).columns.tolist()

tg_primary = 'Failure_Score'
tg_secondary = 'Failure_Category'
ft_base = ['Flowline_Age', 'MAXOPPRESSURE', 'oil_spilled', 'PW_spilled', 'cond_spilled', 'other_spilled', 'MAXOD', 'LAT', 'LONG']
ft_encoded = ['WeatherConditions_Freq_Encoded', 'Root_Cause_Freq_Encoded', 'Spill_Type_Target_Encoded', 'Typeoffluidtrans_Encoded', 'Pipematerial_Freq_Encoded', 'Basin_Freq_Encoded', 'Flowlinetype_Freq_Encoded']
ft_combined = ft_base + ft_encoded

categorical_mapping_config = [
    ('Root Cause Type', 'Root_Cause_Freq_Encoded', 'frequency'),
    ('TYPEOFFLUIDTRANS', 'Typeoffluidtrans_Encoded', 'frequency'),
    ('PIPEMATERIAL', 'Pipematerial_Freq_Encoded', 'frequency'),
    ('Basin', 'Basin_Freq_Encoded', 'frequency'),
    ('FLOWLINETYPE', 'Flowlinetype_Freq_Encoded', 'frequency'),
    ('Weather Conditions', 'WeatherConditions_Freq_Encoded', 'frequency'),
    ('Spill Type_x', 'Spill_Type_Target_Encoded', 'target_mean')
]

def advanced_categorical_encoder(df_input, mapping_config):
    df_output = df_input.copy()
    encoding_stats = {}
    
    for original_col, encoded_col, encoding_type in mapping_config:
        if original_col not in df_output.columns:
            continue
            
        if encoding_type == 'frequency':
            freq_dict = df_output[original_col].value_counts().to_dict()
            df_output[encoded_col] = df_output[original_col].map(freq_dict)
            encoding_stats[encoded_col] = {'type': 'frequency', 'unique_values': len(freq_dict), 'min_freq': min(freq_dict.values()), 'max_freq': max(freq_dict.values())}
            
        elif encoding_type == 'target_mean':
            if tg_primary in df_output.columns:
                target_mean_dict = df_output.groupby(original_col)[tg_primary].mean(numeric_only=True).to_dict()
                df_output[encoded_col] = df_output[original_col].map(target_mean_dict)
                encoding_stats[encoded_col] = {'type': 'target_mean', 'unique_values': len(target_mean_dict)}
    
    return df_output, encoding_stats

df_work, encoding_statistics = advanced_categorical_encoder(df_work, categorical_mapping_config)

def weather_categorization_advanced(condition_str):
    if pd.isna(condition_str):
        return 'Unknown'
    condition_lower = str(condition_str).lower()
    clear_indicators = ['clear', 'sunny', 'bright', 'cloudless']
    rain_indicators = ['rain', 'wet', 'drizzle', 'shower', 'precipitation']
    snow_indicators = ['snow', 'freezing', 'frost', 'ice', 'blizzard']
    cloud_indicators = ['cloudy', 'overcast', 'hazy', 'foggy', 'misty']
    wind_indicators = ['wind', 'gusty', 'breezy', 'storm']
    
    condition_scores = {
        'Clear': sum(1 for indicator in clear_indicators if indicator in condition_lower),
        'Rainy': sum(1 for indicator in rain_indicators if indicator in condition_lower),
        'Snowy': sum(1 for indicator in snow_indicators if indicator in condition_lower),
        'Cloudy': sum(1 for indicator in cloud_indicators if indicator in condition_lower),
        'Windy': sum(1 for indicator in wind_indicators if indicator in condition_lower)
    }
    
    max_score_category = max(condition_scores, key=condition_scores.get)
    return max_score_category if condition_scores[max_score_category] > 0 else 'Other'

df_work['Weather Conditions'] = df_work['Weather Conditions'].apply(weather_categorization_advanced)

interaction_feature_configs = [
    ('Flowline_Age', 'PIPEMATERIAL', 'Age_Material_Interaction', 'string_concat'),
    ('MAXOPPRESSURE', 'TYPEOFFLUIDTRANS', 'Pressure_Type_Interaction', 'string_concat'),
    ('Flowline_Age', 'MAXOPPRESSURE', 'Age_Pressure_Product', 'multiply'),
    ('oil_spilled', 'PW_spilled', 'Total_Liquid_Spilled', 'add'),
    ('LAT', 'LONG', 'Geographic_Distance', 'euclidean')
]

def create_interaction_features(df_input, feature_configs):
    df_output = df_input.copy()
    interaction_stats = {}
    
    for feat1, feat2, new_feat, operation in feature_configs:
        if feat1 not in df_output.columns or feat2 not in df_output.columns:
            continue
            
        if operation == 'string_concat':
            df_output[new_feat] = df_output[feat1].astype(str) + '_' + df_output[feat2].astype(str)
        elif operation == 'multiply':
            df_output[new_feat] = df_output[feat1] * df_output[feat2]
        elif operation == 'add':
            df_output[new_feat] = df_output[feat1] + df_output[feat2]
        elif operation == 'euclidean':
            df_output[new_feat] = np.sqrt((df_output[feat1] - df_output[feat1].mean())**2 + (df_output[feat2] - df_output[feat2].mean())**2)
        
        interaction_stats[new_feat] = {'operation': operation, 'source_features': [feat1, feat2]}
    
    return df_output, interaction_stats

df_work, interaction_statistics = create_interaction_features(df_work, interaction_feature_configs)

binning_configurations = [
    ('MAXOPPRESSURE', 'MAXOPPRESSURE_BIN', 5, ['Very Low', 'Low', 'Medium', 'High', 'Very High']),
    ('Flowline_Age', 'Flowline_Age_BIN', 5, ['Very New', 'New', 'Middle-Aged', 'Old', 'Very Old']),
    ('MAXOD', 'MAXOD_BIN', 4, ['Small', 'Medium', 'Large', 'Very Large']),
    ('oil_spilled', 'Oil_Spill_BIN', 3, ['Low', 'Medium', 'High'])
]

def create_binned_features(df_input, binning_configs):
    df_output = df_input.copy()
    binning_stats = {}
    
    for col, new_col, n_bins, labels in binning_configs:
        if col not in df_output.columns:
            continue
            
        try:
            df_output[new_col] = pd.cut(df_output[col], bins=n_bins, labels=labels)
            binning_stats[new_col] = {'original_column': col, 'bins': n_bins, 'labels': labels}
        except Exception as e:
            print(f"Binning failed for {col}: {e}")
            continue
    
    return df_output, binning_stats

df_work, binning_statistics = create_binned_features(df_work, binning_configurations)

categorical_columns_extended = ['Flowline_Age_BIN', 'Weather Conditions', 'PIPEMATERIAL', 'TYPEOFFLUIDTRANS', 'MAXOPPRESSURE_BIN', 'Root Cause Type', 'Basin', 'Spill Type_x', 'Age_Material_Interaction', 'Pressure_Type_Interaction']
for col in categorical_columns_extended:
    if col in df_work.columns:
        df_work[col] = df_work[col].astype(str)

numerical_columns_extended = ['MAXOD', 'Age_Pressure_Product', 'Total_Liquid_Spilled', 'Geographic_Distance']
for col in numerical_columns_extended:
    if col in df_work.columns:
        df_work[col] = pd.to_numeric(df_work[col], errors='coerce')

failure_categorization_schemes = {
    'conservative': (0.10, 0.80),
    'balanced': (0.20, 0.70),
    'aggressive': (0.35, 0.65),
    'moderate_low': (0.15, 0.75),
    'moderate_high': (0.25, 0.75)
}

def create_multiple_categorizations(df_input, target_col, schemes):
    df_output = df_input.copy()
    categorization_stats = {}
    
    for scheme_name, (low_thresh, high_thresh) in schemes.items():
        low_val = df_output[target_col].quantile(low_thresh)
        high_val = df_output[target_col].quantile(high_thresh)
        
        def categorize_score(score):
            if pd.isna(score):
                return 'Unknown'
            return 'Low' if score < low_val else ('Medium' if score < high_val else 'High')
        
        category_col = f'Failure_Category_{scheme_name}'
        df_output[category_col] = df_output[target_col].apply(categorize_score)
        
        categorization_stats[category_col] = {
            'scheme': scheme_name,
            'thresholds': (low_thresh, high_thresh),
            'values': (low_val, high_val),
            'distribution': df_output[category_col].value_counts().to_dict()
        }
    
    return df_output, categorization_stats

df_work, categorization_stats = create_multiple_categorizations(df_work, tg_primary, failure_categorization_schemes)

feature_sets_config = {
    'basic': ft_combined,
    'extended': ft_combined + ['Age_Material_Interaction', 'Pressure_Type_Interaction', 'Age_Pressure_Product'],
    'binned': ['Flowline_Age_BIN', 'Weather Conditions', 'PIPEMATERIAL', 'TYPEOFFLUIDTRANS', 'MAXOPPRESSURE_BIN', 'MAXOD', 'Root Cause Type', 'Basin', 'Spill Type_x', 'Age_Material_Interaction', 'Pressure_Type_Interaction'],
    'hybrid': ft_combined + ['Flowline_Age_BIN', 'MAXOPPRESSURE_BIN', 'Age_Material_Interaction', 'Total_Liquid_Spilled'],
    'comprehensive': ft_combined + ['Age_Material_Interaction', 'Pressure_Type_Interaction', 'Age_Pressure_Product', 'Total_Liquid_Spilled', 'Geographic_Distance', 'Flowline_Age_BIN', 'MAXOPPRESSURE_BIN']
}

preprocessing_pipeline_configs = {
    'lr_standard': {
        'numerical': pl([('imputer', si(strategy='mean')), ('scaler', ss())]),
        'categorical': pl([('imputer', si(strategy='most_frequent')), ('encoder', ohe(handle_unknown='ignore'))])
    },
    'lr_robust': {
        'numerical': pl([('imputer', ki(n_neighbors=5)), ('scaler', rs())]),
        'categorical': pl([('imputer', si(strategy='most_frequent')), ('encoder', ohe(handle_unknown='ignore'))])
    },
    'rf_basic': {
        'numerical': pl([('imputer', si(strategy='median')), ('scaler', ss())]),
        'categorical': pl([('imputer', si(strategy='most_frequent')), ('encoder', ohe(drop='first', handle_unknown='ignore'))])
    },
    'rf_advanced': {
        'numerical': pl([('imputer', ki(n_neighbors=3)), ('scaler', rs()), ('pca', pca(n_components=0.95))]),
        'categorical': pl([('imputer', si(strategy='most_frequent')), ('encoder', ohe(drop='first', handle_unknown='ignore'))])
    },
    'ann_deep': {
        'numerical': pl([('imputer', si(strategy='mean')), ('scaler', mms()), ('power', pt())]),
        'categorical': pl([('imputer', si(strategy='most_frequent')), ('encoder', ohe(handle_unknown='ignore'))])
    },
    'svm_polynomial': {
        'numerical': pl([('imputer', ki(n_neighbors=5)), ('scaler', ss()), ('poly', pf(degree=2, interaction_only=True))]),
        'categorical': pl([('imputer', si(strategy='most_frequent')), ('encoder', ohe(handle_unknown='ignore'))])
    },
    'svm_kernel': {
        'numerical': pl([('imputer', si(strategy='median')), ('scaler', rs()), ('quantile', qt(n_quantiles=100))]),
        'categorical': pl([('imputer', si(strategy='most_frequent')), ('encoder', ohe(handle_unknown='ignore'))])
    },
    'ensemble_stacking': {
        'numerical': pl([('imputer', ki(n_neighbors=7)), ('scaler', ss()), ('feature_select', skb(fc, k=20))]),
        'categorical': pl([('imputer', si(strategy='most_frequent')), ('encoder', ohe(handle_unknown='ignore'))])
    },
    'ensemble_voting': {
        'numerical': pl([('imputer', si(strategy='mean')), ('scaler', rs()), ('pca', pca(n_components=15))]),
        'categorical': pl([('imputer', si(strategy='most_frequent')), ('encoder', ohe(handle_unknown='ignore'))])
    }
}

def create_preprocessor(config_name, numerical_features, categorical_features):
    if config_name not in preprocessing_pipeline_configs:
        config_name = 'lr_standard'
    
    config = preprocessing_pipeline_configs[config_name]
    return ct(transformers=[
        ('num', config['numerical'], numerical_features),
        ('cat', config['categorical'], categorical_features)
    ])

resampling_strategies = {
    'smote_basic': sm(random_state=42),
    'smote_borderline': bsm(random_state=42, kind='borderline-1'),
    'smote_svm': ssm(random_state=42),
    'adasyn': ad(random_state=42),
    'smote_tomek': smt(random_state=42),
    'smote_enn': sen(random_state=42),
    'random_under': rus(random_state=42)
}

def apply_resampling(X, y, strategy_name):
    if strategy_name not in resampling_strategies:
        strategy_name = 'smote_basic'
    
    try:
        resampler = resampling_strategies[strategy_name]
        return resampler.fit_resample(X, y)
    except Exception as e:
        print(f"Resampling {strategy_name} failed: {e}, using basic SMOTE")
        return sm(random_state=42).fit_resample(X, y)

model_configurations = {
    'lr_basic': {
        'model': lr(random_state=42),
        'params': {'C': [0.01, 0.1, 1, 10], 'penalty': ['l1', 'l2'], 'solver': ['liblinear', 'saga']},
        'features': 'basic',
        'target': 'Failure_Category_conservative',
        'preprocessor': 'lr_standard',
        'resampling': 'smote_basic'
    },
    'lr_advanced': {
        'model': lr(random_state=42, max_iter=1000),
        'params': {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2', 'elasticnet'], 'l1_ratio': [0.1, 0.5, 0.9], 'solver': ['saga']},
        'features': 'extended',
        'target': 'Failure_Category_balanced',
        'preprocessor': 'lr_robust',
        'resampling': 'smote_borderline'
    },
    'rf_basic': {
        'model': rfc(random_state=42, class_weight='balanced'),
        'params': {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, 20], 'min_samples_split': [2, 5, 10]},
        'features': 'binned',
        'target': 'Failure_Category_balanced',
        'preprocessor': 'rf_basic',
        'resampling': 'smote_basic'
    },
    'rf_advanced': {
        'model': rfc(random_state=42, class_weight='balanced_subsample'),
        'params': {'n_estimators': [100, 200, 300], 'max_depth': [10, 20, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'max_features': ['sqrt', 'log2', None]},
        'features': 'comprehensive',
        'target': 'Failure_Category_moderate_low',
        'preprocessor': 'rf_advanced',
        'resampling': 'adasyn'
    },
    'svm_rbf': {
        'model': svc(probability=True, kernel='rbf', random_state=42, class_weight='balanced'),
        'params': {'C': [0.1, 1, 10, 100], 'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1]},
        'features': 'basic',
        'target': 'Failure_Category_conservative',
        'preprocessor': 'svm_kernel',
        'resampling': 'smote_svm'
    },
    'svm_poly': {
        'model': svc(probability=True, kernel='poly', random_state=42, class_weight='balanced'),
        'params': {'C': [0.1, 1, 10], 'degree': [2, 3, 4], 'gamma': ['scale', 'auto']},
        'features': 'extended',
        'target': 'Failure_Category_aggressive',
        'preprocessor': 'svm_polynomial',
        'resampling': 'smote_tomek'
    },
    'ann_shallow': {
        'model': 'neural_network',
        'architecture': [(64, 'relu'), (32, 'relu')],
        'params': {'batch_size': [16, 32, 64], 'epochs': [50, 100, 150], 'learning_rate': [0.001, 0.01]},
        'features': 'hybrid',
        'target': 'Failure_Category_balanced',
        'preprocessor': 'ann_deep',
        'resampling': 'smote_basic'
    },
    'ann_deep': {
        'model': 'neural_network',
        'architecture': [(128, 'relu'), (64, 'relu'), (32, 'relu'), (16, 'relu')],
        'params': {'batch_size': [32, 64], 'epochs': [100, 200], 'learning_rate': [0.0001, 0.001]},
        'features': 'comprehensive',
        'target': 'Failure_Category_moderate_high',
        'preprocessor': 'ann_deep',
        'resampling': 'adasyn'
    }
}

ensemble_configurations = {
    'voting_soft': {
        'type': 'voting',
        'base_models': ['lr_basic', 'rf_basic', 'svm_rbf'],
        'voting': 'soft',
        'features': 'extended',
        'target': 'Failure_Category_balanced',
        'preprocessor': 'ensemble_voting',
        'resampling': 'smote_basic'
    },
    'voting_hard': {
        'type': 'voting',
        'base_models': ['lr_advanced', 'rf_advanced', 'svm_poly'],
        'voting': 'hard',
        'features': 'comprehensive',
        'target': 'Failure_Category_moderate_low',
        'preprocessor': 'ensemble_voting',
        'resampling': 'adasyn'
    },
    'stacking_basic': {
        'type': 'stacking',
        'base_models': [
            rfc(n_estimators=100, random_state=42),
            gbc(n_estimators=100, learning_rate=0.1, random_state=42),
            svc(probability=True, random_state=42)
        ],
        'meta_model': lr(random_state=42),
        'features': 'hybrid',
        'target': 'Failure_Category_balanced',
        'preprocessor': 'ensemble_stacking',
        'resampling': 'smote_tomek'
    },
    'stacking_advanced': {
        'type': 'stacking',
        'base_models': [
            rfc(n_estimators=200, max_depth=10, random_state=42),
            etc(n_estimators=200, max_depth=15, random_state=42),
            gbc(n_estimators=100, learning_rate=0.05, max_depth=5, random_state=42),
            xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
        ],
        'meta_model': lr(C=10, random_state=42),
        'features': 'comprehensive',
        'target': 'Failure_Category_moderate_high',
        'preprocessor': 'ensemble_stacking',
        'resampling': 'smote_enn'
    }
}

visualization_color_schemes = {
    'publication': {
        'colors': ["#ffffff", "#e0e0e0", "#c0c0c0", "#909090", "#606060", "#303030"],
        'risk_colors': {'High': '#800000', 'Medium': '#F0A202', 'Low': '#0D47A1'},
        'line_styles': {'High': '-', 'Medium': '--', 'Low': '-.'},
        'line_widths': {'High': 3.0, 'Medium': 2.5, 'Low': 2.0}
    },
    'colorblind_friendly': {
        'colors': ["#ffffff", "#f0f0f0", "#d9d9d9", "#bdbdbd", "#969696", "#636363"],
        'risk_colors': {'High': '#e31a1c', 'Medium': '#ff7f00', 'Low': '#1f78b4'},
        'line_styles': {'High': '-', 'Medium': '--', 'Low': '-.'},
        'line_widths': {'High': 3.0, 'Medium': 2.5, 'Low': 2.0}
    },
    'grayscale': {
        'colors': ["#ffffff", "#f7f7f7", "#cccccc", "#969696", "#636363", "#252525"],
        'risk_colors': {'High': '#252525', 'Medium': '#636363', 'Low': '#969696'},
        'line_styles': {'High': '-', 'Medium': '--', 'Low': '-.'},
        'line_widths': {'High': 3.0, 'Medium': 2.5, 'Low': 2.0}
    }
}

ordered_classes = ['High', 'Medium', 'Low']

def create_advanced_confusion_matrix(y_true, y_pred, model_name, color_scheme='publication'):
    cm_matrix = cm(y_true, y_pred)
    original_classes = np.unique(np.concatenate([y_true, y_pred]))
    
    try:
        indices = [np.where(original_classes == cls)[0][0] for cls in ordered_classes if cls in original_classes]
        reordered_cm = cm_matrix[indices, :][:, indices] if len(indices) == len(ordered_classes) else cm_matrix
        display_labels = [ordered_classes[i] for i in range(len(indices))] if len(indices) < len(ordered_classes) else ordered_classes
    except:
        reordered_cm = cm_matrix
        display_labels = original_classes
    
    scheme = visualization_color_schemes[color_scheme]
    cmap = lscm.from_list("publication_cmap", scheme['colors'], N=256)
    
    fig_sizes = [(6, 5), (8, 6), (7, 5.5)]
    for i, fig_size in enumerate(fig_sizes):
        fig, ax = plt.subplots(figsize=fig_size, dpi=600)
        
        conf_display = cmd(confusion_matrix=reordered_cm, display_labels=display_labels)
        conf_display.plot(ax=ax, cmap=cmap, colorbar=False, values_format='d', 
                         text_kw={"weight": "bold", "fontsize": 14, "color": "black"})
        
        ax.set_xlabel('Predicted label', fontsize=14, fontweight='bold', labelpad=10)
        ax.set_ylabel('True label', fontsize=14, fontweight='bold', labelpad=10)
        
        for spine in ax.spines.values():
            spine.set_linewidth(1.5)
            spine.set_color('black')
        
        plt.tight_layout()
        
        file_suffix = f"_size{i+1}" if i > 0 else ""
        for ext in ['pdf', 'png', 'svg', 'tiff']:
            filename = f"confusion_matrix_{model_name}_{color_scheme}{file_suffix}.{ext}"
            if ext == 'tiff':
                plt.savefig(filename, format=ext, dpi=600, bbox_inches='tight', pil_kwargs={"compression": "lzw"})
            else:
                plt.savefig(filename, format=ext, dpi=600, bbox_inches='tight')
        
        plt.show()
        plt.close()

def create_advanced_roc_curves(y_true, y_prob, model_obj, model_name, color_scheme='publication'):
    scheme = visualization_color_schemes[color_scheme]
    
    if hasattr(model_obj, 'classes_'):
        classes = model_obj.classes_
    else:
        classes = np.unique(y_true)
    
    class_indices = {cls: i for i, cls in enumerate(classes)}
    
    fig_configs = [
        {'size': (6, 6), 'suffix': ''},
        {'size': (8, 8), 'suffix': '_large'},
        {'size': (10, 8), 'suffix': '_extra_large'}
    ]
    
    for config in fig_configs:
        fig, ax = plt.subplots(figsize=config['size'], dpi=600)
        
        workbook = xlsxwriter.Workbook(f'roc_data_{model_name}{config["suffix"]}.xlsx')
        worksheet = workbook.add_worksheet()
        row = 0
        
        for class_label in ordered_classes:
            if class_label in class_indices and y_prob.shape[1] > class_indices[class_label]:
                idx = class_indices[class_label]
                fpr, tpr, _ = rc(y_true == class_label, y_prob[:, idx])
                roc_auc = auc_fn(fpr, tpr)
                
                ax.plot(fpr, tpr, 
                       lw=scheme['line_widths'][class_label], 
                       color=scheme['risk_colors'][class_label],
                       linestyle=scheme['line_styles'][class_label],
                       label=f'{class_label} Risk (AUC = {roc_auc:.3f})',
                       marker='o', markersize=2, markevery=max(1, len(fpr)//20))
                
                worksheet.write(row, 0, f'Class {class_label} FPR')
                worksheet.write(row, 1, f'Class {class_label} TPR')
                worksheet.write(row, 2, f'AUC = {roc_auc:.3f}')
                for j in range(len(fpr)):
                    worksheet.write(row + j + 1, 0, fpr[j])
                    worksheet.write(row + j + 1, 1, tpr[j])
                row += len(fpr) + 3
        
        workbook.close()
        
        ax.plot([0, 1], [0, 1], color='#FF0000', linestyle='--', lw=2.0, alpha=0.8, label='Random Chance')
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate', fontsize=14, fontweight='bold', labelpad=10)
        ax.set_ylabel('True Positive Rate', fontsize=14, fontweight='bold', labelpad=10)
        ax.grid(True, linestyle='--', alpha=0.3)
        
        handles, labels = ax.get_legend_handles_labels()
        risk_handles = [h for h, l in zip(handles, labels) if 'Random' not in l]
        risk_labels = [l for l in labels if 'Random' not in l]
        
        order_mapping = {f'{cls} Risk (AUC = ': i for i, cls in enumerate(ordered_classes)}
        sorted_indices = sorted(range(len(risk_labels)), 
                              key=lambda i: next((idx for prefix, idx in order_mapping.items() 
                                                 if risk_labels[i].startswith(prefix)), 999))
        
        risk_handles = [risk_handles[i] for i in sorted_indices]
        risk_labels = [risk_labels[i] for i in sorted_indices]
        
        if 'Random Chance' in labels:
            random_idx = labels.index('Random Chance')
            risk_handles.append(handles[random_idx])
            risk_labels.append('Random Chance')
        
        ax.legend(risk_handles, risk_labels, loc='lower right', fontsize=12, frameon=True, 
                 framealpha=0.9, edgecolor='black', facecolor='white')
        
        ax.tick_params(axis='both', which='major', width=1.5, length=6, labelsize=12)
        
        for spine in ax.spines.values():
            spine.set_linewidth(1.5)
            spine.set_color('black')
        
        plt.tight_layout()
        
        for ext in ['pdf', 'png', 'svg', 'tiff']:
            filename = f"roc_curve_{model_name}_{color_scheme}{config['suffix']}.{ext}"
            if ext == 'tiff':
                plt.savefig(filename, format=ext, dpi=600, bbox_inches='tight', pil_kwargs={"compression": "lzw"})
            else:
                plt.savefig(filename, format=ext, dpi=600, bbox_inches='tight')
        
        plt.show()
        plt.close()

def create_neural_network_model(input_dim, output_dim, architecture):
    model = mdl.Sequential()
    model.add(lyr.Dense(architecture[0][0], input_dim=input_dim, activation=architecture[0][1]))
    model.add(lyr.Dropout(0.3))
    
    for layer_size, activation in architecture[1:]:
        model.add(lyr.Dense(layer_size, activation=activation))
        model.add(lyr.Dropout(0.2))
    
    model.add(lyr.Dense(output_dim, activation='softmax'))
    
    optimizer_configs = [
        opt.Adam(learning_rate=0.001),
        opt.Adam(learning_rate=0.0001),
        opt.RMSprop(learning_rate=0.001),
        opt.SGD(learning_rate=0.01, momentum=0.9)
    ]
    
    model.compile(optimizer=optimizer_configs[0], loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def create_residual_analysis_plot(y_true, y_prob, model_name):
    label_encoder = le()
    y_true_encoded = label_encoder.fit_transform(y_true)
    
    residual_errors = []
    class_residuals = {cls: [] for cls in ordered_classes}
    
    for true_label, probabilities in zip(y_true_encoded, y_prob):
        residual_error = 1 - probabilities[true_label]
        residual_errors.append(residual_error)
        
        true_class_name = label_encoder.classes_[true_label]
        if true_class_name in class_residuals:
            class_residuals[true_class_name].append(residual_error)
    
    residual_df = pd.DataFrame({'Residual Error': residual_errors})
    
    fig_configs = [
        {'size': (6, 5), 'bins': 30, 'suffix': ''},
        {'size': (8, 6), 'bins': 50, 'suffix': '_detailed'},
        {'size': (10, 8), 'bins': 20, 'suffix': '_summary'}
    ]
    
    for config in fig_configs:
        fig, axes = plt.subplots(2, 2, figsize=config['size'], dpi=600)
        axes = axes.ravel()
        
        sns.histplot(residual_df['Residual Error'], bins=config['bins'], kde=True, color="#1E88E5", 
                    stat="density", ax=axes[0], alpha=0.7, edgecolor='black', linewidth=0.8)
        axes[0].set_xlabel('Residual Error', fontsize=12, fontweight='bold')
        axes[0].set_ylabel('Density', fontsize=12, fontweight='bold')
        axes[0].set_title('Overall Distribution', fontsize=14, fontweight='bold')
        axes[0].grid(axis='y', linestyle='--', alpha=0.3)
        
        colors = ['#e31a1c', '#ff7f00', '#1f78b4']
        for i, (cls, errors) in enumerate(class_residuals.items()):
            if errors and i < 3:
                sns.histplot(errors, bins=config['bins']//2, kde=True, color=colors[i], 
                           alpha=0.6, ax=axes[i+1], stat="density")
                axes[i+1].set_xlabel('Residual Error', fontsize=12, fontweight='bold')
                axes[i+1].set_ylabel('Density', fontsize=12, fontweight='bold')
                axes[i+1].set_title(f'{cls} Class Residuals', fontsize=14, fontweight='bold')
                axes[i+1].grid(axis='y', linestyle='--', alpha=0.3)
        
        for ax in axes:
            for spine in ax.spines.values():
                spine.set_linewidth(1.5)
                spine.set_color('black')
        
        plt.tight_layout()
        
        for ext in ['pdf', 'png', 'svg']:
            filename = f"residual_analysis_{model_name}{config['suffix']}.{ext}"
            plt.savefig(filename, format=ext, dpi=600, bbox_inches='tight')
        
        plt.show()
        plt.close()

def create_learning_curves(model, X, y, model_name):
    train_sizes = np.linspace(0.1, 1.0, 10)
    
    try:
        train_sizes_abs, train_scores, val_scores = lc(model, X, y, train_sizes=train_sizes, cv=5, 
                                                      scoring='accuracy', n_jobs=-1, random_state=42)
        
        train_mean = np.mean(train_scores, axis=1)
        train_std = np.std(train_scores, axis=1)
        val_mean = np.mean(val_scores, axis=1)
        val_std = np.std(val_scores, axis=1)
        
        fig, ax = plt.subplots(figsize=(8, 6), dpi=600)
        
        ax.plot(train_sizes_abs, train_mean, 'o-', color='#1f77b4', label='Training Score', linewidth=2)
        ax.fill_between(train_sizes_abs, train_mean - train_std, train_mean + train_std, alpha=0.2, color='#1f77b4')
        
        ax.plot(train_sizes_abs, val_mean, 'o-', color='#ff7f0e', label='Validation Score', linewidth=2)
        ax.fill_between(train_sizes_abs, val_mean - val_std, val_mean + val_std, alpha=0.2, color='#ff7f0e')
        
        ax.set_xlabel('Training Set Size', fontsize=14, fontweight='bold')
        ax.set_ylabel('Accuracy Score', fontsize=14, fontweight='bold')
        ax.set_title(f'Learning Curves - {model_name}', fontsize=16, fontweight='bold')
        ax.legend(fontsize=12)
        ax.grid(True, alpha=0.3)
        
        for spine in ax.spines.values():
            spine.set_linewidth(1.5)
            spine.set_color('black')
        
        plt.tight_layout()
        
        for ext in ['pdf', 'png']:
            plt.savefig(f"learning_curves_{model_name}.{ext}", format=ext, dpi=600, bbox_inches='tight')
        
        plt.show()
        plt.close()
        
    except Exception as e:
        print(f"Learning curve creation failed for {model_name}: {e}")

# Model training
def execute_comprehensive_model_training():
    training_results = {}
    model_performances = {}
    training_logs = []
    
    print(f"Starting comprehensive training at {datetime.datetime.now()}")
    
    for config_name, config in model_configurations.items():
        print(f"\n{'='*60}")
        print(f"Training {config_name.upper()}")
        print(f"{'='*60}")
        
        try:
            feature_set = feature_sets_config[config['features']]
            target_column = config['target']
            
            available_features = [f for f in feature_set if f in df_work.columns]
            if not available_features:
                print(f"No features available for {config_name}")
                continue
            
            X = df_work[available_features].copy()
            y = df_work[target_column].copy()
            
            missing_data_summary = X.isnull().sum()
            if missing_data_summary.sum() > 0:
                print(f"Missing data detected: {missing_data_summary.sum()} total missing values")
            
            X = X.dropna()
            y = y[X.index]
            
            if len(X) == 0:
                print(f"No data remaining after cleaning for {config_name}")
                continue
            
            numerical_features = [f for f in available_features if f in df_work.select_dtypes(include=[np.number]).columns]
            categorical_features = [f for f in available_features if f not in numerical_features]
            
            preprocessor = create_preprocessor(config['preprocessor'], numerical_features, categorical_features)
            
            X_preprocessed = preprocessor.fit_transform(X)
            
            if np.isnan(X_preprocessed).any():
                X_preprocessed = np.nan_to_num(X_preprocessed, nan=0.0)
            
            X_resampled, y_resampled = apply_resampling(X_preprocessed, y, config['resampling'])
            
            X_train, X_test, y_train, y_test = tts(X_resampled, y_resampled, test_size=0.2, 
                                                  random_state=42, stratify=y_resampled)
            
            training_start = time.time()
            
            if config['model'] == 'neural_network':
                y_train_onehot = pd.get_dummies(y_train)
                y_test_onehot = pd.get_dummies(y_test)
                
                model = create_neural_network_model(X_train.shape[1], y_train_onehot.shape[1], config['architecture'])
                
                callbacks_list = [
                    es(monitor='val_loss', patience=15, restore_best_weights=True),
                    rlp(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-7),
                    mcp(f'best_model_{config_name}.h5', save_best_only=True)
                ]
                
                history = model.fit(X_train, y_train_onehot, 
                                  epochs=config['params']['epochs'][0],
                                  batch_size=config['params']['batch_size'][0],
                                  validation_split=0.2,
                                  callbacks=callbacks_list,
                                  verbose=1)
                
                y_pred_proba = model.predict(X_test)
                y_pred = np.argmax(y_pred_proba, axis=1)
                y_test_classes = np.argmax(y_test_onehot.values, axis=1)
                
                fig, axes = plt.subplots(2, 2, figsize=(12, 10), dpi=600)
                axes = axes.ravel()
                
                axes[0].plot(history.history['loss'], linewidth=2.5, color='#D81B60', label='Training Loss')
                axes[0].plot(history.history['val_loss'], linewidth=2.5, color='#1E88E5', label='Validation Loss')
                axes[0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
                axes[0].set_ylabel('Loss', fontsize=12, fontweight='bold')
                axes[0].set_title('Loss Curves', fontsize=14, fontweight='bold')
                axes[0].legend()
                axes[0].grid(True, alpha=0.3)
                
                axes[1].plot(history.history['accuracy'], linewidth=2.5, color='#4CAF50', label='Training Accuracy')
                axes[1].plot(history.history['val_accuracy'], linewidth=2.5, color='#FF9800', label='Validation Accuracy')
                axes[1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
                axes[1].set_ylabel('Accuracy', fontsize=12, fontweight='bold')
                axes[1].set_title('Accuracy Curves', fontsize=14, fontweight='bold')
                axes[1].legend()
                axes[1].grid(True, alpha=0.3)
                
                learning_rate_schedule = [float(callbacks_list[1].get_monitor_value(logs={'val_loss': 0.5})) for _ in range(len(history.history['loss']))]
                axes[2].plot(learning_rate_schedule, linewidth=2, color='#9C27B0', label='Learning Rate')
                axes[2].set_xlabel('Epoch', fontsize=12, fontweight='bold')
                axes[2].set_ylabel('Learning Rate', fontsize=12, fontweight='bold')
                axes[2].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
                axes[2].legend()
                axes[2].grid(True, alpha=0.3)
                
                epoch_times = np.diff([0] + list(range(len(history.history['loss']))))
                axes[3].bar(range(len(epoch_times)), epoch_times, color='#607D8B', alpha=0.7)
                axes[3].set_xlabel('Epoch', fontsize=12, fontweight='bold')
                axes[3].set_ylabel('Time (relative)', fontsize=12, fontweight='bold')
                axes[3].set_title('Training Time per Epoch', fontsize=14, fontweight='bold')
                axes[3].grid(True, alpha=0.3)
                
                for ax in axes:
                    for spine in ax.spines.values():
                        spine.set_linewidth(1.5)
                        spine.set_color('black')
                
                plt.tight_layout()
                
                for ext in ['pdf', 'png', 'svg']:
                    plt.savefig(f"training_history_{config_name}.{ext}", format=ext, dpi=600, bbox_inches='tight')
                
                plt.show()
                plt.close()
                
                create_advanced_confusion_matrix(y_test_classes, y_pred, config_name, 'publication')
                create_advanced_roc_curves(y_test_classes, y_pred_proba, model, config_name, 'publication')
                create_residual_analysis_plot(y_test, y_pred_proba, config_name)
                
                best_model = model
                y_test_for_eval = y_test_classes
                y_pred_for_eval = y_pred
                y_pred_proba_for_eval = y_pred_proba
                
            else:
                if hasattr(config['model'], 'fit'):
                    model = config['model']
                    param_grid = config['params']
                    
                    grid_search = gscv(model, param_grid, cv=skf(n_splits=5, shuffle=True, random_state=42), 
                                     scoring='f1_weighted', n_jobs=-1, verbose=1)
                    
                    grid_search.fit(X_train, y_train)
                    best_model = grid_search.best_estimator_
                    
                    print(f"Best parameters for {config_name}: {grid_search.best_params_}")
                    print(f"Best cross-validation score: {grid_search.best_score_:.4f}")
                    
                    cv_results_df = pd.DataFrame(grid_search.cv_results_)
                    cv_results_df.to_excel(f'cv_results_{config_name}.xlsx', engine='xlsxwriter', index=False)
                
                y_pred_for_eval = best_model.predict(X_test)
                y_pred_proba_for_eval = best_model.predict_proba(X_test) if hasattr(best_model, 'predict_proba') else None
                y_test_for_eval = y_test
                
                create_advanced_confusion_matrix(y_test_for_eval, y_pred_for_eval, config_name, 'publication')
                
                if y_pred_proba_for_eval is not None:
                    create_advanced_roc_curves(y_test_for_eval, y_pred_proba_for_eval, best_model, config_name, 'publication')
                    create_residual_analysis_plot(y_test_for_eval, y_pred_proba_for_eval, config_name)
                
                create_learning_curves(best_model, X_resampled, y_resampled, config_name)
            
            training_end = time.time()
            training_duration = training_end - training_start
            
            performance_metrics = {
                'accuracy': acc(y_test_for_eval, y_pred_for_eval),
                'precision_macro': ps(y_test_for_eval, y_pred_for_eval, average='macro'),
                'recall_macro': rs_score(y_test_for_eval, y_pred_for_eval, average='macro'),
                'f1_macro': f1(y_test_for_eval, y_pred_for_eval, average='macro'),
                'f1_weighted': f1(y_test_for_eval, y_pred_for_eval, average='weighted'),
                'training_time': training_duration
            }
            
            if y_pred_proba_for_eval is not None:
                try:
                    performance_metrics['log_loss'] = ll(pd.get_dummies(y_test_for_eval), y_pred_proba_for_eval)
                    performance_metrics['roc_auc_macro'] = ras(pd.get_dummies(y_test_for_eval), y_pred_proba_for_eval, average='macro', multi_class='ovr')
                except:
                    pass
            
            print(f"\n{config_name.upper()} Performance Metrics:")
            for metric, value in performance_metrics.items():
                print(f"{metric}: {value:.4f}" if isinstance(value, float) else f"{metric}: {value}")
            
            classification_rep = cr(y_test_for_eval, y_pred_for_eval, output_dict=True)
            classification_df = pd.DataFrame(classification_rep).transpose()
            classification_df.to_excel(f'classification_report_{config_name}.xlsx', engine='xlsxwriter')
            
            joblib.dump({
                'model': best_model,
                'preprocessor': preprocessor,
                'feature_names': available_features,
                'performance_metrics': performance_metrics,
                'config': config
            }, f'complete_model_{config_name}.pkl')
            
            training_results[config_name] = {
                'model': best_model,
                'preprocessor': preprocessor,
                'performance': performance_metrics,
                'predictions': y_pred_for_eval,
                'probabilities': y_pred_proba_for_eval,
                'test_labels': y_test_for_eval
            }
            
            model_performances[config_name] = performance_metrics
            
            training_log = {
                'model_name': config_name,
                'timestamp': datetime.datetime.now().isoformat(),
                'duration': training_duration,
                'accuracy': performance_metrics['accuracy'],
                'f1_weighted': performance_metrics['f1_weighted'],
                'status': 'success'
            }
            training_logs.append(training_log)
            
            print(f"\n{config_name} training completed successfully!")
            
        except Exception as e:
            error_log = {
                'model_name': config_name,
                'timestamp': datetime.datetime.now().isoformat(),
                'error': str(e),
                'status': 'failed'
            }
            training_logs.append(error_log)
            print(f"Error training {config_name}: {e}")
            continue
    
    training_summary_df = pd.DataFrame(training_logs)
    training_summary_df.to_excel('training_summary.xlsx', engine='xlsxwriter', index=False)
    
    performance_comparison_df = pd.DataFrame(model_performances).transpose()
    performance_comparison_df.to_excel('model_performance_comparison.xlsx', engine='xlsxwriter')
    
    return training_results, model_performances, training_logs

def execute_ensemble_model_training():
    ensemble_results = {}
    
    print(f"\n{'='*60}")
    print("TRAINING ENSEMBLE MODELS")
    print(f"{'='*60}")
    
    for ensemble_name, ensemble_config in ensemble_configurations.items():
        print(f"\nTraining {ensemble_name}...")
        
        try:
            feature_set = feature_sets_config[ensemble_config['features']]
            target_column = ensemble_config['target']
            
            available_features = [f for f in feature_set if f in df_work.columns]
            X = df_work[available_features].dropna()
            y = df_work[target_column][X.index]
            
            numerical_features = [f for f in available_features if f in df_work.select_dtypes(include=[np.number]).columns]
            categorical_features = [f for f in available_features if f not in numerical_features]
            
            preprocessor = create_preprocessor(ensemble_config['preprocessor'], numerical_features, categorical_features)
            X_preprocessed = preprocessor.fit_transform(X)
            
            X_resampled, y_resampled = apply_resampling(X_preprocessed, y, ensemble_config['resampling'])
            X_train, X_test, y_train, y_test = tts(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)
            
            if ensemble_config['type'] == 'voting':
                base_estimators = []
                for base_model_name in ensemble_config['base_models']:
                    if base_model_name in model_configurations:
                        base_config = model_configurations[base_model_name]
                        model_instance = base_config['model']
                        base_estimators.append((base_model_name, model_instance))
                
                ensemble_model = vc_cls(estimators=base_estimators, voting=ensemble_config['voting'])
                
            elif ensemble_config['type'] == 'stacking':
                ensemble_model = sc(
                    estimators=[(f'base_{i}', model) for i, model in enumerate(ensemble_config['base_models'])],
                    final_estimator=ensemble_config['meta_model'],
                    cv=skf(n_splits=5, shuffle=True, random_state=42)
                )
            
            ensemble_model.fit(X_train, y_train)
            
            y_pred = ensemble_model.predict(X_test)
            y_pred_proba = ensemble_model.predict_proba(X_test) if hasattr(ensemble_model, 'predict_proba') else None
            
            performance_metrics = {
                'accuracy': acc(y_test, y_pred),
                'f1_weighted': f1(y_test, y_pred, average='weighted'),
                'f1_macro': f1(y_test, y_pred, average='macro')
            }
            
            print(f"{ensemble_name} Performance:")
            for metric, value in performance_metrics.items():
                print(f"{metric}: {value:.4f}")
            
            create_advanced_confusion_matrix(y_test, y_pred, ensemble_name, 'publication')
            if y_pred_proba is not None:
                create_advanced_roc_curves(y_test, y_pred_proba, ensemble_model, ensemble_name, 'publication')
            
            joblib.dump({
                'model': ensemble_model,
                'preprocessor': preprocessor,
                'performance': performance_metrics
            }, f'ensemble_model_{ensemble_name}.pkl')
            
            ensemble_results[ensemble_name] = {
                'model': ensemble_model,
                'performance': performance_metrics
            }
            
        except Exception as e:
            print(f"Ensemble {ensemble_name} training failed: {e}")
            continue
    
    return ensemble_results

def generate_comprehensive_analysis_report(training_results, ensemble_results):
    print(f"\n{'='*80}")
    print("GENERATING COMPREHENSIVE ANALYSIS REPORT")
    print(f"{'='*80}")
    
    all_results = {**training_results, **ensemble_results}
    
    performance_summary = []
    for model_name, result in all_results.items():
        if 'performance' in result:
            perf = result['performance']
            summary_row = {
                'Model': model_name,
                'Accuracy': perf.get('accuracy', 0),
                'F1_Weighted': perf.get('f1_weighted', 0),
                'F1_Macro': perf.get('f1_macro', 0),
                'Training_Time': perf.get('training_time', 0)
            }
            performance_summary.append(summary_row)
    
    performance_df = pd.DataFrame(performance_summary)
    performance_df = performance_df.sort_values('F1_Weighted', ascending=False)
    
    performance_df.to_excel('final_performance_summary.xlsx', engine='xlsxwriter', index=False)
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12), dpi=600)
    
    axes[0, 0].bar(performance_df['Model'], performance_df['Accuracy'], color='skyblue', alpha=0.7)
    axes[0, 0].set_title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('Accuracy', fontweight='bold')
    axes[0, 0].tick_params(axis='x', rotation=45)
    
    axes[0, 1].bar(performance_df['Model'], performance_df['F1_Weighted'], color='lightgreen', alpha=0.7)
    axes[0, 1].set_title('F1-Weighted Score Comparison', fontsize=14, fontweight='bold')
    axes[0, 1].set_ylabel('F1-Weighted', fontweight='bold')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    axes[1, 0].bar(performance_df['Model'], performance_df['F1_Macro'], color='orange', alpha=0.7)
    axes[1, 0].set_title('F1-Macro Score Comparison', fontsize=14, fontweight='bold')
    axes[1, 0].set_ylabel('F1-Macro', fontweight='bold')
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    training_times = performance_df[performance_df['Training_Time'] > 0]
    if not training_times.empty:
        axes[1, 1].bar(training_times['Model'], training_times['Training_Time'], color='red', alpha=0.7)
        axes[1, 1].set_title('Training Time Comparison', fontsize=14, fontweight='bold')
        axes[1, 1].set_ylabel('Time (seconds)', fontweight='bold')
        axes[1, 1].tick_params(axis='x', rotation=45)
    
    for ax in axes.flat:
        ax.grid(True, alpha=0.3)
        for spine in ax.spines.values():
            spine.set_linewidth(1.5)
    
    plt.tight_layout()
    
    for ext in ['pdf', 'png', 'svg']:
        plt.savefig(f'model_comparison_summary.{ext}', format=ext, dpi=600, bbox_inches='tight')
    
    plt.show()
    plt.close()
    
    best_model_name = performance_df.iloc[0]['Model']
    print(f"\nBest performing model: {best_model_name}")
    print(f"Best F1-Weighted score: {performance_df.iloc[0]['F1_Weighted']:.4f}")
    
    end_time = time.time()
    total_duration = end_time - start_time
    
    final_report = {
        'experiment_id': dt_hash,
        'total_duration': total_duration,
        'models_trained': len(all_results),
        'best_model': best_model_name,
        'best_score': performance_df.iloc[0]['F1_Weighted'],
        'timestamp': datetime.datetime.now().isoformat()
    }
    
    with open('experiment_summary.json', 'w') as f:
        json.dump(final_report, f, indent=2)
    
    print(f"\nExperiment completed in {total_duration:.2f} seconds")
    print(f"Total models trained: {len(all_results)}")
    print("\nFiles generated:")
    
    file_list = [
        'final_performance_summary.xlsx',
        'model_comparison_summary.pdf',
        'experiment_summary.json'
    ]
    
    for model_name in all_results.keys():
        file_list.extend([
            f'confusion_matrix_{model_name}_publication.pdf',
            f'roc_curve_{model_name}_publication.pdf',
            f'classification_report_{model_name}.xlsx',
            f'complete_model_{model_name}.pkl'
        ])
    
    for file_name in file_list:
        if os.path.exists(file_name):
            print(f"✓ {file_name}")
    
    return final_report

# System execution
if __name__ == "__main__":
    print("="*80)
    print("COMPREHENSIVE MACHINE LEARNING PIPELINE EXECUTION")
    print("="*80)
    print(f"Data hash: {dt_hash}")
    print(f"Start time: {datetime.datetime.now()}")
    
    try:
        gc.collect()
        
        print("\nPhase 1: Individual Model Training")
        training_results, model_performances, training_logs = execute_comprehensive_model_training()
        
        print("\nPhase 2: Ensemble Model Training") 
        ensemble_results = execute_ensemble_model_training()
        
        print("\nPhase 3: Comprehensive Analysis")
        final_report = generate_comprehensive_analysis_report(training_results, ensemble_results)
        
        print("\n" + "="*80)
        print("EXPERIMENT COMPLETED SUCCESSFULLY")
        print("="*80)
        
        successful_models = len([log for log in training_logs if log.get('status') == 'success'])
        failed_models = len([log for log in training_logs if log.get('status') == 'failed'])
        
        print(f"Successful models: {successful_models}")
        print(f"Failed models: {failed_models}")
        print(f"Total execution time: {final_report['total_duration']:.2f} seconds")
        print(f"Best model: {final_report['best_model']}")
        print(f"Best score: {final_report['best_score']:.4f}")
        
    except Exception as e:
        print(f"CRITICAL ERROR: {str(e)}")
        import traceback
        traceback.print_exc()
        print("\nDiagnostic Information:")
        print(f"Python version: {sys.version}")
        print(f"Working directory: {os.getcwd()}")
        print(f"Data file exists: {os.path.exists(fp_main)}")
        print(f"Available memory: {gc.get_stats()}")

### Updated ANN_SVM Model

In [ ]:
import os
[os.environ.update({k: '1'}) for k in ['OMP_NUM_THREADS','OPENBLAS_NUM_THREADS','MKL_NUM_THREADS','VECLIB_MAXIMUM_THREADS','NUMEXPR_NUM_THREADS','TF_NUM_INTEROP_THREADS','TF_NUM_INTRAOP_THREADS']]

import pandas as pd, numpy as np, tensorflow as tf
from tensorflow.keras import layers as lyr, models as mdl, regularizers as reg, callbacks as cb
from sklearn.model_selection import train_test_split as tts, GridSearchCV as gscv, StratifiedKFold as skf
from sklearn.preprocessing import StandardScaler as ss, OneHotEncoder as ohe, LabelEncoder as le, RobustScaler as rs, PowerTransformer as pt
from sklearn.compose import ColumnTransformer as ct
from sklearn.pipeline import Pipeline as pl
from sklearn.svm import SVC as svc
from sklearn.ensemble import RandomForestClassifier as rfc, GradientBoostingClassifier as gbc
from sklearn.metrics import classification_report as cr, roc_curve as rc, auc as auc_fn, confusion_matrix as cm, f1_score as f1, accuracy_score as acc_s, balanced_accuracy_score as ba, cohen_kappa_score as ck, matthews_corrcoef as mc
from sklearn.linear_model import LogisticRegression as lr
from imblearn.over_sampling import SMOTE as sm, ADASYN as ad, BorderlineSMOTE as bsm
from imblearn.under_sampling import EditedNearestNeighbours as enn, TomekLinks as tl
from imblearn.combine import SMOTETomek as smt
from imblearn.pipeline import Pipeline as ipl
from sklearn.base import BaseEstimator as be, ClassifierMixin as cmx
from sklearn.feature_selection import SelectKBest as skb, f_classif as fc, RFE as rfe
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap as lscm
import seaborn as sns
import json, joblib, xlsxwriter, warnings, sklearn
warnings.filterwarnings('ignore')

pf = "plots"
if not os.path.exists(pf): os.makedirs(pf); print(f"Created plots folder: {pf}")
sv = sklearn.__version__; print(f"Using scikit-learn version: {sv}")

def get_ohe(**kwargs):
    vm, vn = map(int, sv.split('.')[:2])
    return ohe(sparse_output=False, **kwargs) if vm > 1 or (vm == 1 and vn >= 2) else ohe(sparse=False, **kwargs)

def safe_smote_resample(X, y, random_state=42):
    try:
        smote = sm(random_state=random_state, k_neighbors=min(5, len(np.unique(y))-1), n_jobs=1)
        X_resampled, y_resampled = smote.fit_resample(X, y)
        print("Successfully applied SMOTE resampling"); return X_resampled, y_resampled
    except Exception as e:
        print(f"SMOTE failed: {e}")
        try:
            from imblearn.over_sampling import RandomOverSampler as ros
            ros_obj = ros(random_state=random_state)
            X_resampled, y_resampled = ros_obj.fit_resample(X, y)
            print("Applied RandomOverSampler as fallback"); return X_resampled, y_resampled
        except Exception as e2:
            print(f"RandomOverSampler also failed: {e2}"); print("Using original data without resampling")
            return X, y

plt.rcParams.update({
    'font.family': 'Arial', 'font.size': 11, 'font.weight': 'bold', 'axes.linewidth': 1.2,
    'axes.labelsize': 14, 'axes.labelweight': 'bold', 'axes.titlesize': 16, 'axes.titleweight': 'bold',
    'xtick.major.width': 1.2, 'ytick.major.width': 1.2, 'xtick.major.size': 5, 'ytick.major.size': 5,
    'xtick.labelsize': 12, 'ytick.labelsize': 12, 'legend.frameon': False, 'legend.fontsize': 11,
    'figure.dpi': 600, 'savefig.dpi': 600, 'savefig.bbox': 'tight', 'savefig.pad_inches': 0.1,
})

def load_and_preprocess_data(fp):
    df = pd.read_excel(fp)
    tgt = 'Failure_Score'
    fts = ['Flowline_Age', 'MAXOPPRESSURE', 'oil_spilled', 'PW_spilled', 'cond_spilled', 'other_spilled', 
           'MAXOD', 'LAT', 'LONG', 'WeatherConditions_Freq_Encoded', 'Root_Cause_Freq_Encoded', 
           'Spill_Type_Target_Encoded', 'Typeoffluidtrans_Encoded', 'Pipematerial_Freq_Encoded', 
           'Basin_Freq_Encoded', 'Flowlinetype_Freq_Encoded']
    
    print("Applying frequency encoding...")
    rcf = df['Root Cause Type'].value_counts().to_dict(); df['Root_Cause_Freq_Encoded'] = df['Root Cause Type'].map(rcf)
    ttf = df['TYPEOFFLUIDTRANS'].value_counts().to_dict(); df['Typeoffluidtrans_Encoded'] = df['TYPEOFFLUIDTRANS'].map(ttf)
    pmf = df['PIPEMATERIAL'].value_counts().to_dict(); df['Pipematerial_Freq_Encoded'] = df['PIPEMATERIAL'].map(pmf)
    bf = df['Basin'].value_counts().to_dict(); df['Basin_Freq_Encoded'] = df['Basin'].map(bf)
    ftf = df['FLOWLINETYPE'].value_counts().to_dict(); df['Flowlinetype_Freq_Encoded'] = df['FLOWLINETYPE'].map(ftf)
    wcf = df['Weather Conditions'].value_counts().to_dict(); df['WeatherConditions_Freq_Encoded'] = df['Weather Conditions'].map(wcf)
    
    print("Applying target encoding...")
    sttm = df.groupby('Spill Type_x')[tgt].mean(numeric_only=True).to_dict()
    df['Spill_Type_Target_Encoded'] = df['Spill Type_x'].map(sttm)
    
    print("Creating failure categories...")
    lt, ht = df[tgt].quantile(0.35), df[tgt].quantile(0.70)
    cs = lambda s: 'Low' if s < lt else ('Medium' if s < ht else 'High')
    df['Failure_Category'] = df[tgt].apply(cs)
    df = df.dropna(subset=[tgt] + fts + ['Failure_Category'])
    
    print(f"Data preprocessing completed. Final dataset shape: {df.shape}")
    print(f"Failure distribution:\n{df['Failure_Category'].value_counts()}")
    
    return df, fts, tgt

class ANNSVMEnsemble(be, cmx):
    def __init__(self, em='weighted_voting', aw=0.6, sw=0.4, vb=True):
        self.em, self.aw, self.sw, self.vb = em, aw, sw, vb
        self.am, self.sm, self.ml, self.pp, self.le = None, None, None, None, None
        self.cn = ['High', 'Medium', 'Low']
        
    def bam(self, id, od):
        m = mdl.Sequential([lyr.Dense(128, input_dim=id, activation='relu'), lyr.Dropout(0.3),
                           lyr.Dense(64, activation='relu'), lyr.Dropout(0.3),
                           lyr.Dense(32, activation='relu'), lyr.Dropout(0.3),
                           lyr.Dense(od, activation='softmax')])
        m.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']); return m
    
    def bsp(self):
        return svc(probability=True, kernel='rbf', random_state=42, class_weight='balanced')
    
    def fit(self, Xt, yt, vs=0.2):
        if self.vb: print("Training Original ANN-SVM Ensemble...")
        
        self.le = le(); ye = self.le.fit_transform(yt)
        
        cf = ['WeatherConditions_Freq_Encoded', 'Root_Cause_Freq_Encoded', 
              'Spill_Type_Target_Encoded', 'Typeoffluidtrans_Encoded', 
              'Pipematerial_Freq_Encoded', 'Basin_Freq_Encoded', 'Flowlinetype_Freq_Encoded']
        nf = ['Flowline_Age', 'MAXOPPRESSURE', 'oil_spilled', 'PW_spilled', 'cond_spilled', 
              'other_spilled', 'MAXOD', 'LAT', 'LONG']
        
        self.pp = ct(transformers=[('num', ss(), nf), ('cat', get_ohe(handle_unknown='ignore'), cf)])
        Xp = self.pp.fit_transform(Xt)
        
        Xab, yab = safe_smote_resample(Xp, ye)
        yao = tf.keras.utils.to_categorical(yab, num_classes=len(self.cn))
        Xat, Xav, yat, yav = tts(Xab, yao, test_size=vs, random_state=42, stratify=yab)
        
        id, od = Xat.shape[1], len(self.cn)
        self.am = self.bam(id, od)
        es = cb.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        self.am.fit(Xat, yat, epochs=100, batch_size=32, validation_data=(Xav, yav),
                   callbacks=[es], verbose=1 if self.vb else 0)
        
        Xsb, ysb = safe_smote_resample(Xp, ye)
        sbm = self.bsp()
        pg = {'C': [0.1, 1, 10], 'gamma': [0.1, 1, 10], 'kernel': ['rbf']}
        sg = gscv(sbm, pg, cv=5, scoring='accuracy', n_jobs=1, verbose=0)
        sg.fit(Xsb, ysb); self.sm = sg.best_estimator_
        
        return None
    
    def predict_proba(self, X):
        Xp = self.pp.transform(X)
        ap = self.am.predict(Xp, verbose=0); sp = self.sm.predict_proba(Xp)
        sc = self.sm.classes_; aco = [0, 1, 2]
        
        if not np.array_equal(sc, aco):
            ri = [np.where(sc == i)[0][0] for i in aco]; sp = sp[:, ri]
        
        ep = self.aw * ap + self.sw * sp; return ep
    
    def predict(self, X):
        pb = self.predict_proba(X); return np.argmax(pb, axis=1)
    
    def save_models(self, bp):
        self.am.save(f"{bp}_ann_model.h5"); joblib.dump(self.sm, f"{bp}_svm_model.pkl")
        joblib.dump(self.pp, f"{bp}_preprocessor.pkl"); joblib.dump(self.le, f"{bp}_label_encoder.pkl")
        print(f"Ensemble models saved with base path: {bp}")

# Processing workflow
class AdvancedANNSVMEnsemble(be, cmx):
    def __init__(self, em='adaptive_weighted', uaf=True, ufl=True, vb=True):
        self.em, self.uaf, self.ufl, self.vb = em, uaf, ufl, vb
        self.am, self.sm, self.rm, self.ml, self.pp, self.le, self.fs = [None]*7
        self.cn, self.mw = ['High', 'Medium', 'Low'], {'ann': 0.4, 'svm': 0.4, 'rf': 0.2}

    def fl(self, a=0.25, g=2.0):
        def flf(yt, yp):
            yp = tf.clip_by_value(yp, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
            ce = -yt * tf.math.log(yp); w = a * yt * tf.math.pow((1 - yp), g)
            fl = w * ce; return tf.reduce_mean(tf.reduce_sum(fl, axis=1))
        return flf

    def baam(self, id, od):
        m = mdl.Sequential([
            lyr.Dense(256, input_dim=id), lyr.BatchNormalization(), lyr.Activation('relu'), lyr.Dropout(0.4),
            lyr.Dense(128, kernel_regularizer=reg.l2(0.001)), lyr.BatchNormalization(), lyr.Activation('relu'), lyr.Dropout(0.3),
            lyr.Dense(64, kernel_regularizer=reg.l2(0.001)), lyr.BatchNormalization(), lyr.Activation('relu'), lyr.Dropout(0.2),
            lyr.Dense(od, activation='softmax')
        ])
        
        lf = self.fl(alpha=0.25, gamma=2.0) if self.ufl else 'categorical_crossentropy'
        m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=lf, metrics=['accuracy']); return m

    def caf(self, X):
        if not self.uaf: return X
        Xe = X.copy()
        
        if 'Flowline_Age' in X.columns and 'MAXOPPRESSURE' in X.columns:
            Xe['Age_Pressure_Interaction'] = X['Flowline_Age'] * X['MAXOPPRESSURE']
            r = X['Flowline_Age'] / (X['MAXOPPRESSURE'] + 1e-8)
            Xe['Age_Pressure_Ratio'] = np.where(np.isfinite(r), r, 0)
        
        if all(col in X.columns for col in ['oil_spilled', 'PW_spilled', 'cond_spilled', 'other_spilled']):
            os = X['oil_spilled'].fillna(0); ps = X['PW_spilled'].fillna(0)
            cs = X['cond_spilled'].fillna(0); ots = X['other_spilled'].fillna(0)
            Xe['Total_Spill_Volume'] = os + ps + cs + ots
            Xe['Spill_Diversity_Index'] = ((os > 0).astype(int) + (ps > 0).astype(int) + 
                                          (cs > 0).astype(int) + (ots > 0).astype(int))
        
        if 'Flowline_Age' in X.columns:
            as_ = X['Flowline_Age'].fillna(0)
            Xe['Age_Squared'] = as_ ** 2; Xe['Age_Log'] = np.log1p(np.maximum(as_, 0))
        
        Xe = Xe.replace([np.inf, -np.inf], 0); return Xe

    def fit(self, Xt, yt, vs=0.2):
        if self.vb: print("Training Advanced ANN-SVM Ensemble...")
        
        Xte = self.caf(Xt)
        if Xte.isnull().any().any():
            print("Warning: NaN values detected after feature engineering. Filling with median/mode...")
            nc = Xte.select_dtypes(include=[np.number]).columns
            Xte[nc] = Xte[nc].fillna(Xte[nc].median())
            cc = Xte.select_dtypes(exclude=[np.number]).columns
            for col in cc:
                Xte[col] = Xte[col].fillna(Xte[col].mode().iloc[0] if not Xte[col].mode().empty else 0)
        
        self.le = le(); ye = self.le.fit_transform(yt)
        cf = [col for col in Xte.columns if 'Encoded' in col or 'Bin' in col]
        nf = [col for col in Xte.columns if col not in cf]
        
        from sklearn.impute import SimpleImputer as si
        self.pp = ct(transformers=[
            ('num', pl([('imputer', si(strategy='median')), ('scaler', rs())]), nf),
            ('cat', pl([('imputer', si(strategy='most_frequent')), ('encoder', get_ohe(handle_unknown='ignore'))]), cf)
        ])
        
        Xp = self.pp.fit_transform(Xte)
        if np.isnan(Xp).any(): print("Warning: NaN values still present after preprocessing. Replacing with zeros..."); Xp = np.nan_to_num(Xp, nan=0.0)
        
        self.fs = skb(score_func=fc, k='all'); Xs = self.fs.fit_transform(Xp, ye)
        
        try:
            rs_obj = smt(smote=sm(random_state=42, k_neighbors=min(5, len(np.unique(ye))-1), n_jobs=1),
                        tomek=tl(n_jobs=1), random_state=42)
            Xr, yr = rs_obj.fit_resample(Xs, ye); print("Applied SMOTETomek resampling")
        except Exception as e:
            print(f"SMOTETomek failed: {e}"); Xr, yr = safe_smote_resample(Xs, ye)
        
        if self.vb: print("Training Advanced ANN model...")
        yao = tf.keras.utils.to_categorical(yr, num_classes=len(self.cn))
        Xat, Xav, yat, yav = tts(Xr, yao, test_size=vs, random_state=42, stratify=yr)
        
        self.am = self.baam(Xat.shape[1], len(self.cn))
        cbs = [cb.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
               cb.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7)]
        self.am.fit(Xat, yat, epochs=150, batch_size=32, validation_data=(Xav, yav),
                   callbacks=cbs, verbose=1 if self.vb else 0)
        
        if self.vb: print("Training Advanced SVM model...")
        sm_obj = svc(probability=True, random_state=42, class_weight='balanced')
        pg = {'C': [0.1, 1, 10, 100], 'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1], 'kernel': ['rbf', 'poly']}
        sg = gscv(sm_obj, pg, cv=5, scoring='f1_weighted', n_jobs=1, verbose=0)
        sg.fit(Xr, yr); self.sm = sg.best_estimator_
        
        if self.vb: print("Training Random Forest for diversity...")
        rm_obj = rfc(n_estimators=200, max_depth=10, min_samples_split=5,
                    min_samples_leaf=2, class_weight='balanced', random_state=42)
        self.rm = rm_obj.fit(Xr, yr)
        
        if self.em in ['adaptive_weighted', 'hierarchical']:
            if self.vb: print("Training meta-learner...")
            ap = self.am.predict(Xr, verbose=0); sp = self.sm.predict_proba(Xr); rp = self.rm.predict_proba(Xr)
            mf = np.hstack([ap, sp, rp])
            ac, sc, rc = np.max(ap, axis=1), np.max(sp, axis=1), np.max(rp, axis=1)
            cf_obj = np.column_stack([ac, sc, rc]); mf = np.hstack([mf, cf_obj])
            self.ml = gbc(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
            self.ml.fit(mf, yr)
        
        if self.vb: print("Advanced ensemble training completed!"); return None

    def predict_proba(self, X):
        Xe = self.caf(X)
        if Xe.isnull().any().any():
            nc = Xe.select_dtypes(include=[np.number]).columns; Xe[nc] = Xe[nc].fillna(Xe[nc].median())
            cc = Xe.select_dtypes(exclude=[np.number]).columns
            for col in cc:
                Xe[col] = Xe[col].fillna(Xe[col].mode().iloc[0] if not Xe[col].mode().empty else 0)
        
        Xp = self.pp.transform(Xe)
        if np.isnan(Xp).any(): Xp = np.nan_to_num(Xp, nan=0.0)
        Xs = self.fs.transform(Xp)
        
        ap = self.am.predict(Xs, verbose=0); sp = self.sm.predict_proba(Xs); rp = self.rm.predict_proba(Xs)
        
        sc, rc, to = self.sm.classes_, self.rm.classes_, [0, 1, 2]
        if not np.array_equal(sc, to):
            ri = [np.where(sc == i)[0][0] for i in to]; sp = sp[:, ri]
        if not np.array_equal(rc, to):
            ri = [np.where(rc == i)[0][0] for i in to]; rp = rp[:, ri]
        
        if self.em == 'adaptive_weighted':
            ac, sc_conf, rc_conf = np.max(ap, axis=1), np.max(sp, axis=1), np.max(rp, axis=1)
            tc = ac + sc_conf + rc_conf
            aw, sw, rw = ac / tc, sc_conf / tc, rc_conf / tc
            ep = (aw[:, np.newaxis] * ap + sw[:, np.newaxis] * sp + rw[:, np.newaxis] * rp)
            
        elif self.em == 'hierarchical' and self.ml is not None:
            mf = np.hstack([ap, sp, rp])
            ac, sc_conf, rc_conf = np.max(ap, axis=1), np.max(sp, axis=1), np.max(rp, axis=1)
            cf_obj = np.column_stack([ac, sc_conf, rc_conf]); mf = np.hstack([mf, cf_obj])
            epe = self.ml.predict_proba(mf)
            ep = np.zeros((len(X), len(self.cn)))
            for i, ci in enumerate(self.ml.classes_): ep[:, ci] = epe[:, i]
        
        else: ep = (self.mw['ann'] * ap + self.mw['svm'] * sp + self.mw['rf'] * rp)
        
        return ep

    def predict(self, X):
        pb = self.predict_proba(X); return np.argmax(pb, axis=1)
    
    def save_models(self, bp):
        self.am.save(f"{bp}_ann_model.h5"); joblib.dump(self.sm, f"{bp}_svm_model.pkl")
        joblib.dump(self.rm, f"{bp}_rf_model.pkl"); joblib.dump(self.pp, f"{bp}_preprocessor.pkl")
        joblib.dump(self.le, f"{bp}_label_encoder.pkl"); joblib.dump(self.fs, f"{bp}_feature_selector.pkl")
        if self.ml: joblib.dump(self.ml, f"{bp}_meta_learner.pkl")
        print(f"Advanced ensemble models saved with base path: {bp}")

def advanced_model_evaluation(ens, Xt, yt, cn=['High', 'Medium', 'Low']):
    ypp = ens.predict_proba(Xt); ypc = np.argmax(ypp, axis=1); yte = ens.le.transform(yt)
    
    acc = acc_s(yte, ypc); ba_val = ba(yte, ypc)
    f1w = f1(yte, ypc, average='weighted'); f1m = f1(yte, ypc, average='macro')
    kp = ck(yte, ypc); mcc_val = mc(yte, ypc)
    
    print(f"\nAdvanced Evaluation Metrics:"); print(f"Accuracy: {acc:.4f}")
    print(f"Balanced Accuracy: {ba_val:.4f}"); print(f"F1-Score (Weighted): {f1w:.4f}")
    print(f"F1-Score (Macro): {f1m:.4f}"); print(f"Cohen's Kappa: {kp:.4f}")
    print(f"Matthews Correlation Coefficient: {mcc_val:.4f}")
    print(f"\nDetailed Classification Report:"); print(cr(yte, ypc, target_names=cn))
    
    return {'accuracy': acc, 'balanced_accuracy': ba_val, 'f1_weighted': f1w, 'f1_macro': f1m, 'kappa': kp, 'mcc': mcc_val}

def run_enhanced_ensemble_comparison():
    print("="*80); print("ENHANCED ANN-SVM ENSEMBLE WITH ADVANCED TECHNIQUES"); print("="*80)
    
    # Data loading pipeline
    ef = "C:/Users/anisi/OneDrive - Colorado School of Mines/Documents/PhD/PhD Research/ECMC/Spills/Spills_Data.xlsx"
    print("Loading and preprocessing ECMC data..."); dt, fts, tgt = load_and_preprocess_data(ef)
    
    X, y = dt[fts], dt['Failure_Category']
    print(f"Dataset shape: {X.shape}"); print(f"Class distribution:\n{y.value_counts()}")
    
    Xt, Xte, yt, yte = tts(X, y, test_size=0.3, random_state=42, stratify=y)
    rc = {}
    
    print("\n" + "="*60); print("TRAINING ORIGINAL ENSEMBLE"); print("="*60)
    oe = ANNSVMEnsemble(em='weighted_voting', vb=True); oe.fit(Xt, yt)
    ypo = oe.predict(Xte); ytee = oe.le.transform(yte); oa = acc_s(ytee, ypo)
    rc['original'] = {'accuracy': oa, 'model': oe, 'predictions': ypo}
    print(f"Original Ensemble Accuracy: {oa:.4f}")
    
    print("\n" + "="*60); print("TRAINING ENHANCED ENSEMBLE WITH FOCAL LOSS"); print("="*60)
    ef_obj = AdvancedANNSVMEnsemble(em='adaptive_weighted', uaf=True, ufl=True, vb=True)
    ef_obj.fit(Xt, yt); emf = advanced_model_evaluation(ef_obj, Xte, yte)
    rc['enhanced_focal'] = {**emf, 'model': ef_obj}
    
    print("\n" + "="*60); print("TRAINING ENHANCED ENSEMBLE WITH HIERARCHICAL METHOD"); print("="*60)
    eh_obj = AdvancedANNSVMEnsemble(em='hierarchical', uaf=True, ufl=True, vb=True)
    eh_obj.fit(Xt, yt); emh = advanced_model_evaluation(eh_obj, Xte, yte)
    rc['enhanced_hierarchical'] = {**emh, 'model': eh_obj}
    
    print("\n" + "="*80); print("COMPREHENSIVE COMPARISON ANALYSIS"); print("="*80)
    
    cdf = pd.DataFrame({
        'Original Ensemble': [rc['original']['accuracy'], 'N/A', 'N/A', 'N/A', 'N/A', 'N/A'],
        'Enhanced + Focal Loss': [rc['enhanced_focal']['accuracy'], rc['enhanced_focal']['balanced_accuracy'],
                                 rc['enhanced_focal']['f1_weighted'], rc['enhanced_focal']['f1_macro'],
                                 rc['enhanced_focal']['kappa'], rc['enhanced_focal']['mcc']],
        'Enhanced + Hierarchical': [rc['enhanced_hierarchical']['accuracy'], rc['enhanced_hierarchical']['balanced_accuracy'],
                                   rc['enhanced_hierarchical']['f1_weighted'], rc['enhanced_hierarchical']['f1_macro'],
                                   rc['enhanced_hierarchical']['kappa'], rc['enhanced_hierarchical']['mcc']]
    }, index=['Accuracy', 'Balanced Accuracy', 'F1-Weighted', 'F1-Macro', 'Cohen\'s Kappa', 'MCC'])
    
    print("\nPerformance Comparison:"); print(cdf.round(4))
    
    fi = ((rc['enhanced_focal']['accuracy'] - rc['original']['accuracy']) / rc['original']['accuracy']) * 100
    hi = ((rc['enhanced_hierarchical']['accuracy'] - rc['original']['accuracy']) / rc['original']['accuracy']) * 100
    
    print(f"\nImprovement Analysis:"); print(f"Enhanced + Focal Loss: {fi:.2f}% improvement")
    print(f"Enhanced + Hierarchical: {hi:.2f}% improvement")
    
    print("\n" + "="*60); print("GENERATING SEPARATE CONFUSION MATRIX PLOTS"); print("="*60)
    
    mds = [('Original', rc['original']['model']), ('Enhanced + Focal', rc['enhanced_focal']['model']),
           ('Enhanced + Hierarchical', rc['enhanced_hierarchical']['model'])]
    acs = [rc['original']['accuracy'], rc['enhanced_focal']['accuracy'], rc['enhanced_hierarchical']['accuracy']]
    
    for idx, (nm, md) in enumerate(mds):
        ypt = md.predict(Xte); ytet = md.le.transform(yte); cm_obj = cm(ytet, ypt)
        
        fig, ax = plt.subplots(figsize=(8, 6), dpi=600)
        cls = ["#ffffff", "#e0e0e0", "#c0c0c0", "#909090", "#606060", "#303030"]
        cmp = lscm.from_list("publication_cmap", cls, N=256)
        
        sns.heatmap(cm_obj, annot=True, fmt='d', cmap=cmp, 
                   xticklabels=['High', 'Medium', 'Low'], yticklabels=['High', 'Medium', 'Low'],
                   ax=ax, cbar=True, cbar_kws={'label': 'Count'})
        
        ax.set_xlabel('Predicted Label', fontsize=14, fontweight='bold')
        ax.set_ylabel('True Label', fontsize=14, fontweight='bold')
        ax.tick_params(axis='both', which='major', labelsize=12); plt.tight_layout()
        
        fb = nm.lower().replace(' + ', '_').replace(' ', '_')
        plt.savefig(os.path.join(pf, f'confusion_matrix_{fb}.pdf'), dpi=600, bbox_inches='tight')
        plt.savefig(os.path.join(pf, f'confusion_matrix_{fb}.png'), dpi=600, bbox_inches='tight'); plt.show()
        print(f"Saved: {pf}/confusion_matrix_{fb}.pdf/.png")
    
    print("\n" + "="*60); print("GENERATING SEPARATE ROC CURVE PLOTS"); print("="*60)
    
    cmp_obj = {'High': '#800000', 'Medium': '#F0A202', 'Low': '#0D47A1'}
    ls_obj = {'High': '-', 'Medium': '--', 'Low': '-.'}
    
    for idx, (nm, md) in enumerate(mds):
        yppt = md.predict_proba(Xte); ytet = md.le.transform(yte)
        fig, ax = plt.subplots(figsize=(8, 6), dpi=600)
        
        for i, cn in enumerate(['High', 'Medium', 'Low']):
            yb = (ytet == i).astype(int); fpr, tpr, _ = rc(yb, yppt[:, i]); ra = auc_fn(fpr, tpr)
            ax.plot(fpr, tpr, color=cmp_obj[cn], linestyle=ls_obj[cn], linewidth=3.0,
                   label=f'{cn} Risk (AUC = {ra:.3f})', marker='o', markersize=4, markevery=0.1)
        
        ax.plot([0, 1], [0, 1], 'r--', alpha=0.6, linewidth=2, label='Random Classifier')
        ax.set_xlim([0.0, 1.0]); ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate', fontsize=14, fontweight='bold')
        ax.set_ylabel('True Positive Rate', fontsize=14, fontweight='bold')
        ax.legend(loc='lower right', fontsize=12, frameon=True, fancybox=True, shadow=True, framealpha=0.9)
        ax.grid(True, alpha=0.3, linestyle='-', linewidth=0.5)
        ax.tick_params(axis='both', which='major', labelsize=12); plt.tight_layout()
        
        fb = nm.lower().replace(' + ', '_').replace(' ', '_')
        plt.savefig(os.path.join(pf, f'roc_curve_{fb}.pdf'), dpi=600, bbox_inches='tight')
        plt.savefig(os.path.join(pf, f'roc_curve_{fb}.png'), dpi=600, bbox_inches='tight'); plt.show()
        print(f"Saved: {pf}/roc_curve_{fb}.pdf/.png")
    
    print("\n" + "="*60); print("GENERATING CLASS-SPECIFIC PERFORMANCE COMPARISON"); print("="*60)
    
    cm_data = {}
    for mn, md_data in rc.items():
        if mn == 'original': continue
        md_obj = md_data['model']; ypt = md_obj.predict(Xte); ytet = md_obj.le.transform(yte)
        crp = cr(ytet, ypt, target_names=['High', 'Medium', 'Low'], output_dict=True)
        cm_data[mn] = {'High': {'precision': crp['High']['precision'], 'recall': crp['High']['recall'], 'f1': crp['High']['f1-score']},
                      'Medium': {'precision': crp['Medium']['precision'], 'recall': crp['Medium']['recall'], 'f1': crp['Medium']['f1-score']},
                      'Low': {'precision': crp['Low']['precision'], 'recall': crp['Low']['recall'], 'f1': crp['Low']['f1-score']}}
    
    mts = ['precision', 'recall', 'f1']; cls_list = ['High', 'Medium', 'Low']
    
    for mt in mts:
        fig, ax = plt.subplots(figsize=(10, 6), dpi=600); x = np.arange(len(cls_list)); w = 0.25
        cls_colors = ['#1f77b4', '#ff7f0e', '#2ca02c']
        
        for i, (mn, dt) in enumerate(cm_data.items()):
            vs = [dt[cls][mt] for cls in cls_list]; lbl = mn.replace('_', ' ').title()
            ax.bar(x + i * w, vs, w, label=lbl, color=cls_colors[i], alpha=0.8)
        
        ax.set_xlabel('Risk Classes', fontsize=14, fontweight='bold')
        ax.set_ylabel(f'{mt.title()} Score', fontsize=14, fontweight='bold')
        ax.set_xticks(x + w); ax.set_xticklabels(cls_list); ax.legend(fontsize=12); ax.grid(True, alpha=0.3, axis='y')
        
        for i, (mn, dt) in enumerate(cm_data.items()):
            vs = [dt[cls][mt] for cls in cls_list]
            for j, v in enumerate(vs):
                ax.text(j + i * w, v + 0.01, f'{v:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=10)
        
        plt.tight_layout()
        plt.savefig(os.path.join(pf, f'class_specific_{mt}_comparison.pdf'), dpi=600, bbox_inches='tight')
        plt.savefig(os.path.join(pf, f'class_specific_{mt}_comparison.png'), dpi=600, bbox_inches='tight'); plt.show()
        print(f"Saved: {pf}/class_specific_{mt}_comparison.pdf/.png")
    
    print("\n" + "="*60); print("FEATURE IMPORTANCE ANALYSIS"); print("="*60)
    
    bmn = max(rc.keys(), key=lambda x: rc[x]['accuracy'] if 'accuracy' in rc[x] else 0)
    bm = rc[bmn]['model']
    
    if hasattr(bm, 'rm') and bm.rm is not None:
        print("Analyzing feature importance from Random Forest component...")
        bf = list(X.columns); is_obj = bm.rm.feature_importances_
        if len(is_obj) > len(bf):
            fn = bf + [f"engineered_feature_{i}" for i in range(len(is_obj) - len(bf))]
        else: fn = bf[:len(is_obj)]
        
        idf = pd.DataFrame({'Feature': fn[:len(is_obj)], 'Importance': is_obj}).sort_values('Importance', ascending=False).head(15)
        fig, ax = plt.subplots(figsize=(12, 8), dpi=600)
        brs = ax.barh(idf['Feature'], idf['Importance'], color='steelblue', alpha=0.8, edgecolor='navy', linewidth=1)
        
        for i, (br, vl) in enumerate(zip(brs, idf['Importance'])):
            ax.text(vl + 0.001, br.get_y() + br.get_height()/2, f'{vl:.3f}', ha='left', va='center', fontweight='bold', fontsize=10)
        
        ax.set_xlabel('Feature Importance Score', fontsize=14, fontweight='bold')
        ax.set_ylabel('Features', fontsize=14, fontweight='bold'); ax.grid(True, alpha=0.3, axis='x'); ax.invert_yaxis()
        plt.tight_layout()
        plt.savefig(os.path.join(pf, 'feature_importance_enhanced.pdf'), dpi=600, bbox_inches='tight')
        plt.savefig(os.path.join(pf, 'feature_importance_enhanced.png'), dpi=600, bbox_inches='tight'); plt.show()
        print("Top 10 Most Important Features:"); print(idf.head(10).to_string(index=False))
        print(f"Saved: {pf}/feature_importance_enhanced.pdf/.png")
    else: print("Feature importance analysis not available")
    
    print("\n" + "="*60); print("SAVING COMPREHENSIVE RESULTS"); print("="*60)
    
    cdf.to_excel('enhanced_ensemble_comparison.xlsx', engine='xlsxwriter')
    dr = {'comparison_summary': cdf.to_dict(), 'improvements': {'focal_loss_improvement': fi, 'hierarchical_improvement': hi},
          'best_model': bmn, 'best_accuracy': rc[bmn]['accuracy'], 'class_specific_metrics': cm_data}
    
    with open('enhanced_ensemble_detailed_results.json', 'w') as f: json.dump(dr, f, indent=2, default=str)
    bm.save_models(f"best_enhanced_ensemble_{bmn}")
    
    print("Results saved:"); print("- enhanced_ensemble_comparison.xlsx")
    print("- enhanced_ensemble_detailed_results.json")
    print(f"- Individual confusion matrices: {pf}/confusion_matrix_[model_name].pdf/.png")
    print(f"- Individual ROC curves: {pf}/roc_curve_[model_name].pdf/.png")
    print(f"- Class-specific performance: {pf}/class_specific_[metric]_comparison.pdf/.png")
    print(f"- {pf}/feature_importance_enhanced.pdf/.png")
    print(f"- best_enhanced_ensemble_{bmn}_*.pkl/h5")
    
    return rc, bmn

def quick_improvement_suggestions():
    sg = [{"technique": "Class Weight Adjustment", "implementation": "Increase penalties for Low risk misclassification", "expected_improvement": "5-10%", "effort": "Low"},
         {"technique": "Advanced Resampling", "implementation": "Use ADASYN or BorderlineSMOTE instead of standard SMOTE", "expected_improvement": "3-7%", "effort": "Low"},
         {"technique": "Feature Engineering", "implementation": "Add interaction terms and polynomial features", "expected_improvement": "5-15%", "effort": "Medium"},
         {"technique": "Ensemble Diversity", "implementation": "Add XGBoost or LightGBM to the ensemble", "expected_improvement": "8-12%", "effort": "Medium"},
         {"technique": "Focal Loss Implementation", "implementation": "Fine-tune alpha and gamma parameters", "expected_improvement": "5-10%", "effort": "Low"},
         {"technique": "Data Preprocessing", "implementation": "Use RobustScaler instead of StandardScaler", "expected_improvement": "2-5%", "effort": "Low"}]
    
    print("\n" + "="*80); print("QUICK IMPROVEMENT SUGGESTIONS (PRIORITIZED)"); print("="*80)
    sgdf = pd.DataFrame(sg); print(sgdf.to_string(index=False)); return sg


# Main execution
if __name__ == "__main__":
    print(f"Python version: {tf.__version__}"); print(f"TensorFlow version: {tf.__version__}"); print(f"Scikit-learn version: {sklearn.__version__}")
    
    try:
        print("Starting enhanced ensemble comparison..."); rs, bm = run_enhanced_ensemble_comparison()
        sg = quick_improvement_suggestions(); print_final_recommendations()
        
        print(f"\n{'='*80}"); print("SUMMARY"); print(f"{'='*80}")
        print(f"Best performing model: {bm}"); print(f"Best accuracy achieved: {rs[bm]['accuracy']:.4f}")
        
        if 'original' in rs:
            imp = ((rs[bm]['accuracy'] - rs['original']['accuracy']) / rs['original']['accuracy']) * 100
            print(f"Improvement over original: {imp:.2f}%")
        
        print("\nNext steps:"); print("1. Review the generated comparison plots and reports")
        print("2. Implement the suggested quick wins"); print("3. Consider domain-specific feature engineering")
        print("4. Test the best model on PHMSA data"); print("5. Iterate based on real-world validation results")
        
    except Exception as e:
        print(f"Error during execution: {str(e)}"); import traceback; traceback.print_exc()
        print("\nTroubleshooting tips:"); print("1. Check your data file path")
        print("2. Ensure all required packages are installed:")
        print("   pip install pandas numpy tensorflow scikit-learn imbalanced-learn matplotlib seaborn xlsxwriter joblib")
        print("3. Update scikit-learn if needed: pip install --upgrade scikit-learn")
        print("4. Reduce model complexity if memory issues occur"); print("5. Try restarting your Python kernel/environment")
        
        print("\nAttempting to create sample plots...")
        try:
            import matplotlib.pyplot as plt, numpy as np
            fig, ax = plt.subplots(figsize=(6, 4)); x = np.linspace(0, 10, 100); y = np.sin(x)
            ax.plot(x, y); ax.set_xlabel('X values'); ax.set_ylabel('Y values'); plt.tight_layout()
            plt.savefig(os.path.join(pf, 'test_plot.png'), dpi=300, bbox_inches='tight'); plt.show()
            print(f"Test plot saved to {pf}/test_plot.png")
        except Exception as pe:
            print(f"Even basic plotting failed: {pe}"); print("There may be a fundamental issue with your Python environment.")